# Coset Characters

This is the development notebook for the library that calculates coset characters and partition functions for $\hat{\text{su}}(2)_k$ WZW models and their cosets

## Characters for $\hat{\text{su}}(2)_k$

We know from amazing Di-Francesco that the coset characters of $\hat{\text{su}}(2)_k$ are given by
$$
\chi_{\lambda}^{(k)} = \frac{\Theta^{(k+2)}_{\lambda + 1} - \Theta^{(k+2)}_{-\lambda - 1}}{\Theta^{(2)}_{1} - \Theta^{(2)}_{-1}}.
$$
Therefore in some way we need to calculate these $\Theta$ functions. Luckily he has also done this for us, so in the $q$ expansion we have that
$$
\chi_{\lambda}^{(k)}(q) = q^{\frac{(\lambda+1)^2}{4(k+2)} - \frac{1}{8}} \frac{\sum_{n\in \mathbb{Z}} [\lambda + 1 + 2n(k+2)] q^{n[\lambda + 1 + n(k+2)]}}{\sum_{n \in \mathbb{Z}} (1+4n)q^{n(1+2n)}}
$$
The first sum is some fractional power that contains the central charge term, but the second is not! The second has integer coefficients! The interesting part is calculating this ratio of sums.

We can quickly show that both of the polynomials are of the form
$$
a(q) = \sum_{n\in \mathbb{Z}_{\geq 0}} \hat a(n) q^{\hat f(n)}, 
$$ 
where $\hat f(2n) \coloneqq f(n)$, $\hat f(2n+1) \coloneqq f(-n)$, and similarly for $\hat a$ where $f(n) = n[\lambda + 1 + n(k+2)]$ is the exponent function over the integers. Not only that, but $\hat f$ is strictly increasing which has some nice properties.

For our numerical evaluation, we effectively have something perfect. We can store everything in a sparse array, and do operations there.

In [1]:
from flint import fmpz_mpoly_ctx     
# (btw#1: fmpz stands for flint multivariate polynomial in Z)
# (btw#2: Flint stores multivariate polynomials sparsely but not univariate)

# Define the context of the polynomials 
ctx = fmpz_mpoly_ctx.get(('q',), 'lex') # Lexicographically ordered polynomials in q

# Use the context to obtain a random polynomial and print it
f = ctx.from_dict({(7,): 5, (0,): 3, (5,): -8})

print(f, '\n', f.to_dict())


5*q^7 - 8*q^5 + 3 
 {(7,): 5, (5,): -8, (0,): 3}


In [2]:
# Define a little printing function for a polynomial up to order n
from IPython.display import display, Latex

def show(f,n:float=100, show:bool=True):
    terms = f.to_dict()
    monomials = []
    for power in reversed(terms):
        if power[0] > n: 
            break
        monomials.append(f'{terms[power]} q^{power[0]}')

    
    string = " +".join(monomials)\
        .replace("+-","-")\
        .replace(" q^0","")
    
    if not show:
        return string
    
    display(Latex("$"+string+"$"))

show(f)
    

<IPython.core.display.Latex object>